In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/home/ryuky/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


labels.csv
sample_submission.csv
test
train



In [2]:
# parameters
tensorboard_dir = '../tb/run1';
tensorboardFlag = True;

batchSize = 64;
epochs = 10;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
df_train = df_train.head(100)
df_test = df_test.head(100)

Training images:  10222
Test images:  10357


In [4]:
df_train.head(10)

id               breed
0  000bec180eb18c7604dcecc8fe0dba07         boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97               dingo
2  001cdf01b096e06d78e9e5112d419397            pekinese
3  00214f311d5d2247d5dfe4fe24b2303d            bluetick
4  0021f9ceb3235effd7fcde7f7538ed62    golden_retriever
5  002211c81b498ef88e1b40b9abf84e1d  bedlington_terrier
6  00290d3e1fdd27226ba27a8ce248ce85  bedlington_terrier
7  002a283a315af96eaea0e28e7163b21b              borzoi
8  003df8b8a8b05244b1d920bb6cf451f9             basenji
9  0042188c895a2f14ef64a918ed9c7b64  scottish_deerhound

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 100/100 [00:00<00:00, 291.38it/s]


In [10]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 100/100 [00:00<00:00, 279.61it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(100, 90, 90, 3)
(100, 62)
(100, 90, 90, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

Number of classes:  62


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = VGG19(#weights='imagenet',
    weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [];
callbacks_list.append(keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=10,
    verbose=1));
if tensorboardFlag:
    callbacks_list.append(keras.callbacks.TensorBoard(
            log_dir=tensorboard_dir,
            histogram_freq=1,
            write_graph=True,
            write_images=True));
    print('Tensorboard activated')
else:
    print('Tensorboard NOT activated')


model.summary()

80142336/80134624 [==============================] - 53s 1us/step
Tensorboard activated
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
______________________________________________________

In [16]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

Train on 70 samples, validate on 30 samples
Epoch 1/10
70/70 [==============================] - 13s 181ms/step - loss: 4.6295 - acc: 0.0143 - val_loss: 4.6914 - val_acc: 0.0333
Epoch 2/10
70/70 [==============================] - 13s 186ms/step - loss: 3.9302 - acc: 0.0429 - val_loss: 4.9551 - val_acc: 0.0000e+00
Epoch 3/10
70/70 [==============================] - 13s 182ms/step - loss: 3.4840 - acc: 0.0857 - val_loss: 5.2571 - val_acc: 0.0000e+00
Epoch 4/10
70/70 [==============================] - 13s 181ms/step - loss: 3.1628 - acc: 0.1143 - val_loss: 5.5536 - val_acc: 0.0000e+00
Epoch 5/10
70/70 [==============================] - 13s 189ms/step - loss: 2.8924 - acc: 0.1429 - val_loss: 5.8005 - val_acc: 0.0000e+00
Epoch 6/10
70/70 [==============================] - 13s 181ms/step - loss: 2.6401 - acc: 0.2714 - val_loss: 5.9906 - val_acc: 0.0000e+00
Epoch 7/10
70/70 [==============================] - 13s 179ms/step - loss: 2.3898 - acc: 0.3857 - val_loss: 6.1461 - val_acc: 0.0000e+00
E

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [17]:
preds = model.predict(x_test, verbose=1)

100/100 [==============================] - 9s 92ms/step


In [18]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(10)

id  affenpinscher  african_hunting_dog  \
0  000621fb3cbb32d8935728e48679680e       0.034822             0.023369   
1  00102ee9d8eb90812350685311fe5890       0.011956             0.038411   
2  0012a730dfa437f5f3613fb75efcd4ce       0.007924             0.081537   
3  001510bc8570bbeee98c8d80c8a95ec1       0.005954             0.057660   
4  001a5f3114548acdefa3d4da05474c2e       0.079452             0.065200   
5  00225dcd3e4d2410dd53239f95c0352f       0.010627             0.024953   
6  002c2a3117c2193b4d26400ce431eebd       0.004839             0.041772   
7  002c58d413a521ae8d1a5daeb35fc803       0.001513             0.001023   
8  002f80396f1e3db687c5932d7978b196       0.015620             0.107993   
9  0036c6bcec6031be9e62a257b1c3c442       0.011943             0.053512   

   appenzeller  australian_terrier   basenji    basset  bedlington_terrier  \
0     0.001218            0.008128  0.004142  0.000019            0.026429   
1     0.008318            0.034900  0.003581  0.000066            0.034309   
2     0.009696            0.008838  0.006484  0.000126            0.045497   
3     0.013951            0.005339  0.007692  0.000102            0.072573   
4     0.002139            0.019739  0.007536  0.000097            0.060161   
5     0.003805            0.013228  0.004301  0.000051            0.123672   
6     0.010505            0.014171  0.004931  0.000146            0.042287   
7     0.001187            0.070144  0.000603  0.000019            0.026298   
8     0.004220            0.027069  0.009699  0.000151            0.033642   
9     0.002051            0.009829  0.003947  0.000033            0.063189   

   black-and-tan_coonhound  bloodhound           ...             \
0                 0.013153    0.008564           ...              
1                 0.010230    0.006517           ...              
2                 0.004156    0.010113           ...              
3                 0.008204    0.018978           ...              
4                 0.003370    0.010055           ...              
5                 0.007277    0.012188           ...              
6                 0.005525    0.024759           ...              
7                 0.010179    0.004295           ...              
8                 0.005737    0.011488           ...              
9                 0.007195    0.006083           ...              

   scottish_deerhound  shetland_sheepdog  shih-tzu  standard_schnauzer  \
0            0.057233           0.000205  0.006516            0.003366   
1            0.031513           0.000424  0.004808            0.009547   
2            0.058236           0.000293  0.007332            0.006129   
3            0.055024           0.000299  0.006915            0.011418   
4            0.030724           0.000355  0.037111            0.006732   
5            0.026335           0.000217  0.015258            0.005302   
6            0.116026           0.001034  0.004148            0.018008   
7            0.041680           0.000133  0.002430            0.003334   
8            0.030035           0.000477  0.012970            0.010804   
9            0.043963           0.000303  0.007270            0.006914   

   tibetan_terrier  toy_poodle  walker_hound  weimaraner  \
0         0.000838    0.008531      0.015289    0.011521   
1         0.009141    0.013460      0.019772    0.005299   
2         0.009211    0.017880      0.046762    0.006554   
3         0.002600    0.013756      0.035258    0.010192   
4         0.002419    0.031522      0.027173    0.014260   
5         0.001011    0.017499      0.022469    0.009869   
6         0.010027    0.013552      0.056465    0.003236   
7         0.001487    0.003805      0.018211    0.001588   
8         0.004357    0.059367      0.012617    0.006945   
9         0.002821    0.020570      0.022229    0.019105   

   welsh_springer_spaniel  wire-haired_fox_terrier  
0                0.007685                 0.015945  
1                0.004815   